<a href="https://colab.research.google.com/github/chrysshawk/Coursera_Capstone/blob/master/IBM_Applied_Data_Science_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IBM Applied Data Science Project: Identifying Business Opportunities


## Table of Contents

1. Problem Introduction
2. Project data collection
3. 

## 1. Problem Introduction




### 1.1 Problem Description and Background

This project will investigate how we can use data about a location to identify business opportunities in that area.

Prior to establishing a new business, one needs to perform market research about the potential areas wherein to establish the business, identifying what the competition is doing, what are the characteristics of the well-performing and poor-performing ventures, as well as identifying any missing market segments.

This project will use the locational data of a given location, investigate the surronding areas and associated businesses, and provide feedback on which business opportunities and which areas will be worth looking into.

### 1.2 Data Description and Use

This section contains a description of which data will be required for the project and how it will be used.

#### 1.2.1 Data need and Collection
The data requirements to carry out this analysis include the following:
1. Data about the zip/postal codes and geolocational position (latitude and longitude) of the surrounding areas
2. Data about the venues we are investigating. These will include:
  - Venue Popularity
  - Venue Price Range (1-4, 1=least expensive, 4=most expensive)
  - Venue Category
  - Venue Postal Code
  - Vene geospatial location
3. Detailed data about the most popular venues, so we can analyze what are their characteristics.
  - Venue xxx
  - Venue yyy
  - Venue zzz

For part (1) we are going to rely on the data and functionality as provided by Geonames's API service. GeoNames API will look up zip/postal code information surronding the selected city/location, including the geospatial data of these zip/postal codes. This also accepts reverse geocoding, that is, the possibility to provide a given set of location information of a given postal code.

Furthermore, Geocoder will be used to get details about given location's coordinates. For the purpose of this project, we specifically use the Arcgis source for the geocoder to ensure the data structure is consistent across potential different locations and countries.

For part (2) we will use Foursquare's [Places API](https://developer.foursquare.com/docs/places-api/endpoints/) to fetch data about the most popular venues in the given area. For this purpose, we will need to set up a free account with Foursquare, which allows for up to 950 regular calls per day. For this task, we will use Foursquare's API GET requests for regular calls to:
- Endpoint group: Venue
- Endpoint: Explore
- [Endpoint documentation](https://developer.foursquare.com/docs/api-reference/venues/explore/)


Given the selected location, we are going to make multiple calls to Foursquare to fetch as many venues as are available (cap at 500) and ask Foursquare to rank them according to popularity. This feature will be essential for our analysis, since we are interested in identifying what characteristics makes for a popular venue. For this purpose we will se the non-premium Foursquare endpoint "explore". Further definition of this can be found at:  

For part (3) we will go a bit deeper and look at further details about the most popular venues, using Foursquare's premium calls, of which we have 50 calls per day as part of the free account. The endpoint we will use for this is "details", whose definition is at: [Endpoint documentation](https://developer.foursquare.com/docs/api-reference/venues/details/)


#### 1.2.2 Transforming data to problem solution

Given we have data about the given area, as well as data about the venues in the given area, we are going to do the following: 
1. Consolidate the venue data we get from Geonames and Foursquare, so we have a list of the venues in the area sorted by popularity
2. Analyze the features of the most popular venue types, identifying where they are located, what type of venues they are, and what price range they are in.
3. Do K-Means clustering on the venues to investigate how machine learning will categorize the features of the popular venue types.
4. Use the above to provide a recommendation for business opportunities, identifying which business would be likely to succeeed, by answering the following:
  - In which area(s) is the venue likely to be popular?
  - What type of venue is likely to be popular?
  - What price category should the venue be?
  - xxx

Given the above recommendation, we should be in a good position to use the gathered information to investigate the possibility of establishing a new business.

## 2. Project data collection

In this section we will collect the required data for the project.

### Importing required libraries for project

In [ ]:
# Defining imports
# importing necessary libraries
import pandas as pd # library for analysis
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import numpy as np # library to handle data

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geopy
# install the Geocoder
!pip -q install geocoder
import geocoder

import requests # library to handle requests
from bs4 import BeautifulSoup # library to process web content
import lxml # libary to parse web content
import urllib.request # import the library we use to open URLs
from pandas.io.json import json_normalize # library to flatten json objects

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Math library for mathematical calculations
import math

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 102kB 3.8MB/s 
Libraries imported.


### Defining Project Input Parameters

#### 1. Which city/location to analyze

In [ ]:
# Defining input city
CITY = 'Las Vegas'
STATE = 'Nevada'
COUNTRY = 'United States'
radius = 15 # radius (km) from location center to include in scope

INPUT_ADDRESS = ', '.join((CITY,STATE,COUNTRY))
print('Location selected for the analysis:', INPUT_ADDRESS)

Location selected for the analysis: Las Vegas, Nevada, United States


#### 2. Credentials for Geonames service and FourSquare

In [ ]:
# Defining user for geonames service
user = 'chauk' # user for geonames service

# Foursquare Parameters
CLIENT_ID = 'xxx' # your Foursquare ID
CLIENT_SECRET = 'xxx' # your Foursquare Secret
CLIENT_ID = 'MVYIIDNKGNQSTQV5O5KK2H4LZNTUA0OT0XFCMOEKNXZ4BOD2' # your Foursquare ID
CLIENT_SECRET = 'EAFFVEBG41P5BNRGICUCAWFNXWFQ5XWXZOAQKSXS04RRN3YE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 50 # limit on how many entries Foursquare will return
RADIUS = radius*1000 # radius in meters
POPULARITY = 1 # boolean meaning venues will be sorted by popularity

SECTIONS = ['food', 'drinks', 'coffee', 'shops', 'arts', 'outdoors', 'sights'] # possible sections/categories
SECTION = SECTIONS[0] # Selecting the Food section

### Retrieving location details and displaying location on map

In [ ]:
# Using the geolocator to identify the geospatial location of selected city
geolocator = Nominatim(user_agent="city_explorer")
location = geolocator.geocode(INPUT_ADDRESS)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of', CITY, 'are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Las Vegas are 36.1672559, -115.1485163.


In [ ]:
# Using Geocoder Arcgis source to look up city
g = geocoder.arcgis(INPUT_ADDRESS)

# Assigning coordinates
latitude, longitude = g.latlng

# Displaying location on map
city_map = folium.Map(location=[latitude, longitude], zoom_start=12)
#city_map

### Retrieving Zip/Postal code details of given area

To do this, we will use the Geonames API.

In [ ]:
# Look up postal codes within given radius of city center
maxRows = 500 # setting parameter of maximum rows to return using the free service

url = 'http://api.geonames.org/findNearbyPostalCodesJSON?lat={}&lng={}&radius={}&maxRows={}&username={}'.format(
    latitude,
    longitude,
    radius,
    maxRows,
    user
)

postal_codes = requests.get(url).json()
print('There are', len(postal_codes['postalCodes']), 'postal codes in', CITY)

There are 71 postal codes in Las Vegas


In [ ]:
# Assigning json results to list, and thereafter to dataframe
location_list = []
location_list.append([(
    pc['postalCode'],
    pc['placeName'],
    pc['adminName1'],
    pc['adminCode1'],
    pc['countryCode'],
    pc['lat'],
    pc['lng']) for pc in postal_codes['postalCodes']])

location_df = pd.DataFrame([entry for location_list in location_list for entry in location_list])
location_df.columns = ['Postal_Code', 'Place_Name', 'State_Name', 'State_Code', 'Country_Code', 'PC_Latitude', 'PC_Longitude']
location_df.sort_values(by='Postal_Code', inplace=True)
location_df.head(5)

,Postal_Code,Place_Name,State_Name,State_Code,Country_Code,PC_Latitude,PC_Longitude
65,89014,Henderson,Nevada,NV,US,36.056435,-115.077968
32,89030,North Las Vegas,Nevada,NV,US,36.211457,-115.124147
53,89031,North Las Vegas,Nevada,NV,US,36.258888,-115.171801
36,89032,North Las Vegas,Nevada,NV,US,36.217968,-115.170919
60,89033,North Las Vegas,Nevada,NV,US,36.284511,-115.134488


Depending on the chosen location and geospatial data accuracy, there might be  postal codes using the same geographical coordinates. To avoid such duplicates, we will group any duplicate geospatial points and merge the postal codes into one row, separated by a comma.

In [ ]:
# grouping postal codes with the same geospatial coordinates
pc_grouped = location_df.groupby(['PC_Latitude', 'PC_Longitude'])['Postal_Code'].transform(lambda x: ', '.join(x))

# making new dataframe based on existing one for postal codes
postal_codes_df = location_df.copy(deep=True)

# overwriting the result back into the dataframe
postal_codes_df['Postal_Code'] = pc_grouped

# since any grouped rows will now be duplicates, we remove the dupes
postal_codes_df.drop_duplicates(inplace=True)

print('We are working with a number of', postal_codes_df.shape[0], 'separate geospatial locations.')

We are working with a number of 46 separate geospatial locations.


To get an idea of the separate geospatial areas, let us plot them into a map.

In [ ]:
# Plotting the relevant areas on the map
city_map = folium.Map(location=[latitude, longitude], 
                      tiles='OpenStreetMap',
                      zoom_start=12)

# Adding markers to map
for lat, lng, label in zip(postal_codes_df['PC_Latitude'], postal_codes_df['PC_Longitude'], postal_codes_df['Postal_Code']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(city_map)  

# Display map of city with the associated labels
city_map

#### Finding out which areas are the most popular

In this section, we will get some insight into which areas are the most popular.

In [ ]:
# Defining function to retrieve venue details from given areas
def getNearbyVenues(postal_codes, latitudes, longitudes, radius, venue_category):
    
    venues_list=[]
    for postal_code, lat, lng in zip(postal_codes, latitudes, longitudes):
        #print(name)
            
        # create the API request URL using the Foursquare format and pre-defined paramenters
        # we select the popularity feature, meaning the results will be ranked in order of most popular
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}&popularity={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            venue_category,
            POPULARITY)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        # since results are ranked, we use enumerate to assign (1-indexed) rank per postal code area
        venues_list.append([(
            postal_code, 
            lat, 
            lng, 
            rank,
            v['venue']['name'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for rank, v in enumerate(results, 1)])

    # Creating a dataframe to store the relevant details of the neighborhoods and venues
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal_Code', 
                  'Postal_Latitude', 
                  'Postal_Longitude', 
                  'Venue_Rank',
                  'Venue_Name', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    
    return(nearby_venues)

In [ ]:
# Making call to the above function to retrieve venues
location_venues = getNearbyVenues(postal_codes=postal_codes_df['Postal_Code'],
                                   latitudes=postal_codes_df['PC_Latitude'],
                                   longitudes=postal_codes_df['PC_Longitude'],
                                   radius = 300, # radius in meters per given postal code
                                   venue_category = SECTION # given venue type
                                  )

In [ ]:
print('Number of venues retrieved:', location_venues.shape[0])
location_venues.loc[location_venues['Venue_Rank']==1].head(5)

Number of venues retrieved: 164


,Postal_Code,Postal_Latitude,Postal_Longitude,Venue_Rank,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
0,89014,36.056435,-115.077968,1,KFC,36.055821,-115.077857,Fast Food Restaurant
1,89031,36.258888,-115.171801,1,Little Taco Stand,36.258045,-115.170603,Mexican Restaurant
2,89033,36.284511,-115.134488,1,In-N-Out Burger,36.285216,-115.134502,Fast Food Restaurant
5,89036,36.198859,-115.117501,1,La Santa Cruz,36.198730,-115.119080,Mexican Restaurant
6,89084,36.281522,-115.148172,1,the grillway,36.280589,-115.147173,Fried Chicken Joint


Ok, so now we have the subset of venues for the relevant areas.

The next step of the analysis is to do some statistics on how many venues are in the various areas; i.e. per given postal code.

In [ ]:
location_venues.groupby('Postal_Code')['Venue_Name'].count().sort_values(ascending=False)[0:10]

Postal_Code
89101           27
89162           22
89119           18
89147           13
89146           10
89117            8
89102            8
89128            6
89105, 89140     6
89169            4
Name: Venue_Name, dtype: int64

Given the above, we get some insight into which areas have the most venues and which are less popular.

We can get some further insight by drilling down into each area by including the category for each venue in the more popular (>5 venues) areas.

In [ ]:
# selecting only the most popular areas
top_number = 5
top_postal_codes = location_venues.groupby('Postal_Code')['Venue_Name'].count().sort_values(ascending=False)[0:top_number].index.values
top_areas = location_venues.loc[location_venues['Postal_Code'].isin(top_postal_codes)]
top_areas

,Postal_Code,Postal_Latitude,Postal_Longitude,Venue_Rank,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
12,89101,36.17193,-115.14001,1,Pizza Rock,36.171707,-115.142343,Pizza Place
13,89101,36.17193,-115.14001,2,Rachel's Kitchen,36.170019,-115.139923,Restaurant
14,89101,36.17193,-115.14001,3,Triple George Grill,36.171804,-115.142159,American Restaurant
15,89101,36.17193,-115.14001,4,Nacho Daddy,36.170316,-115.141619,Mexican Restaurant
16,89101,36.17193,-115.14001,5,Flock & Fowl,36.170334,-115.139672,Asian Restaurant
...,...,...,...,...,...,...,...,...
152,89162,36.17251,-115.14139,18,Lanai Express,36.170653,-115.143773,Chinese Restaurant
153,89162,36.17251,-115.14139,19,Thai Vegas Cuisine,36.170272,-115.141597,Asian Restaurant
154,89162,36.17251,-115.14139,20,322 Pizza Bar,36.170114,-115.141973,Pizza Place
155,89162,36.17251,-115.14139,21,Benny's Smokin' BBQ & Brews,36.171246,-115.144081,BBQ Joint


Let us investigate the number of venues and the mean rating per area. The lower the rating, the more popular are the venues.

In [ ]:
top_areas.groupby(['Postal_Code'])['Venue_Rank'].agg(['count', 'mean'])

,count,mean
Postal_Code,,
89101,27,14.0
89119,18,9.5
89146,10,5.5
89147,13,7.0
89162,22,11.5


**NOTE: THE ABOVE WILL BE SUPER USEFUL IF WE GET THE "REAL" RATINGS IN: IF ONE AREA IS BOTH POPULAR AND HAS POOR RATINGS, WE HAVE A BUSINESS OPPORTUNITY!!!**
However, as it stands, the mean rating will be less useful since it will always increase with the popularity per area.

Let us have a look at the venues with overall highest rating across all areas. To include some statistical detail, we will also include the number of each venue category to ensure there is a sufficient group of venues that are contributing to making the result statistically meaningful.

In [ ]:
top_categories = top_areas.groupby(['Venue_Category'])['Venue_Rank'].agg(['mean', 'count']).sort_values(by='count', ascending=False).head(5).reset_index()
top_categories

,Venue_Category,mean,count
0,American Restaurant,6.818182,11
1,Pizza Place,10.666667,9
2,Fast Food Restaurant,7.875000,8
3,Asian Restaurant,14.000000,7
4,Sandwich Place,12.833333,6


**BUSINESS OPPORTUNITY:** If a venue category is popular despite poor ratings, it is likely that a better venue in that category will be attractive to customers.

In [ ]:
top_categories['Potential'] = top_categories['count'] * top_categories['mean']
top_categories.sort_values(by='Potential', ascending=False, inplace=True)
top_categories.reset_index(drop=True, inplace=True)
top_categories

,Venue_Category,mean,count,Potential
0,Asian Restaurant,14.000000,7,98.0
1,Pizza Place,10.666667,9,96.0
2,Sandwich Place,12.833333,6,77.0
3,American Restaurant,6.818182,11,75.0
4,Fast Food Restaurant,7.875000,8,63.0


In [ ]:
print('The greatest business potential for new Venues would be in the category of', 
      top_categories['Venue_Category'][0], 'and', top_categories['Venue_Category'][1])

The greatest business potential for new Venues would be in the category of Asian Restaurant and Pizza Place


As per the above, we base the business opportunity based on the popularity of the venue vs the rating it gets: The worse rating overall while attracting as high popularity as possible yields the best potential.

#### Strategy A:
- select only one location for returned results, with huge radius (max 100.000m, ie. 100km); maybe 10km, 10.000m
- sort by popularity, and retrieve the info on the first 10 pages (10 calls)
- having all these venues and associated geospatial info, we can merge them into the postal_code overview

### Retrieving all venues in the given area
An important feature of this analysis is to determine the ranking of the various venues in the radius of the given city/location. In order to do this, we are going to use FourSquare's API and ask them to return the results by order of popularity. 

However, since FourSquare only returns 50 results per call, we need to create a function to create a number of calls to access the different result pages. This is done by creating the below function and including the "offset" parameter in the call.

In [ ]:
# Collecting venue ranks across given location

def getVenueRanking(latitude, longitude, radius, venue_category, max_results):
  venues_list = []
  pages = math.ceil(max_results / LIMIT) # calculating the max possible number of FourSquare results pages

  for page in range(0,pages): # looping through each of the results pages (0-indexed)

    # determining rank offset (0-indexed) based on which results page is retrieved
    rank_offset = page * LIMIT
    
    # create the API request URL using the Foursquare format and pre-defined paramenters
    # we select the popularity feature, meaning the results will be ranked in order of most popular
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}&popularity={}&offset={}'.format(
      CLIENT_ID, 
      CLIENT_SECRET, 
      VERSION, 
      latitude, 
      longitude, 
      radius, 
      LIMIT,
      venue_category,
      POPULARITY,
      rank_offset)
    
    # making the GET request
    response = requests.get(url).json()
    total_results = response['response']['totalResults'] # retrieving the total number of results

    # fetching the necessary info from each request
    results = response['response']['groups'][0]['items']

    # return only relevant information for each nearby venue
    # since results are ranked, we use enumerate to assign (1-indexed) rank per postal code area
    venues_list.append([(
      rank,
      v['venue']['id'],
      v['venue']['name'],
      v['venue']['location']['postalCode'],
      v['venue']['location']['lat'], 
      v['venue']['location']['lng'],  
      v['venue']['categories'][0]['name']) for rank, v in enumerate(results, rank_offset)])
    
    # if there are less total results than what we have received
    if (total_results < (LIMIT + page*LIMIT)): 
      break # no more futher API calls
  
  # Creating a dataframe to store the relevant details of the ranked venues
  ranked_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
  ranked_venues.columns = ['Venue_Rank', 
                'Venue_ID',
                'Venue_Name',
                'Venue_Postal_Code',  
                'Venue_Latitude', 
                'Venue_Longitude', 
                'Venue_Category']
  
  return(ranked_venues)

In [ ]:
# Making the call to the function to retrieve the dataframe of ranked venues
max_results = 500
ranked_venues = getVenueRanking(latitude, longitude, RADIUS, SECTION, max_results)
print('Retrieved a total of', ranked_venues.shape[0], 'ranked venues.')

Retrieved a total of 242 ranked venues.


From the above we can get some insight into the distribution of venues across the various postal code areas, as well as the rating. In the above case, the mean is the mean rating, and a lower mean indicates the ratings are higher.

To finish the data retrival, let's create a fuction to see get the associated price ranges of the various venues. 

As above, we first create a function to make the call and structure the data, and second make a call to that function with the required input paramenters.

In [ ]:
# Collecting venue ranks across given location

def getVenuePriceRange(latitude, longitude, radius, venue_category, max_results):
  venues_list = []
  pages = math.ceil(max_results / LIMIT) # calculating the max possible number of FourSquare results pages

  for price in range(1,5): # for each price category (FourSquare ranges from 1-4)
    for page in range(0,pages): # looping through each of the results pages (0-indexed)

      # determining rank offset (0-indexed) based on which results page is retrieved
      rank_offset = page * LIMIT

      # create the API request URL using the Foursquare format and pre-defined paramenters
      # we select the popularity feature, meaning the results will be ranked in order of most popular
      url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}&popularity={}&offset={}&price={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitude, 
        longitude, 
        radius, 
        LIMIT,
        venue_category,
        POPULARITY,
        rank_offset, # this page parameter is for the given page of the result set as defined in the loop
        price) # the price category of the given venue
      
      # determining rank offset (0-indexed) based on which results page is retrieved
      rank_offset = page * LIMIT

      # making the GET request
      response = requests.get(url).json()
      total_results = response['response']['totalResults'] # retrieving the total number of results

      # fetching the necessary info from each request
      results = response['response']['groups'][0]['items']

      # we only want the id of the restaurant and the associated price rante
      venues_list.append([(
        v['venue']['id'],
        price) for v in results])
      
      # if there are less total results than what we have received
      if (total_results < (LIMIT + page*LIMIT)): 
        break # no more futher API calls

  # Creating a dataframe to store the relevant details of the ranked venues
  price_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
  price_venues.columns = ['Venue_ID',
                'Venue_Price_Category']
  
  return(price_venues)

In [ ]:
# Making the call to the function to retrieve the dataframe of ranked venues
price_range_max_results = max_results # for simplicity, reduce the number of pages by number of price ranges
price_range_venues = getVenuePriceRange(latitude, longitude, RADIUS, SECTION, price_range_max_results)
print('Retrieved a total of venues with the given price ranges:', price_range_venues.shape[0])

Retrieved a total of venues with the given price ranges: 240


In [ ]:
price_range_venues.groupby('Venue_Price_Category')['Venue_Price_Category'].agg(['count'])

,count
Venue_Price_Category,
1,69
2,105
3,40
4,26


In [ ]:
print('Unique ranked venues:', ranked_venues.Venue_ID.unique().shape[0])
print('Unique ranked venue names:', ranked_venues.Venue_Name.unique().shape[0])
print('Unique price ranked venues:', price_range_venues.Venue_ID.unique().shape[0])

Unique ranked venues: 242
Unique ranked venue names: 214
Unique price ranked venues: 240


In [ ]:
# Joining the price range information to the venues we already have in our dataset
print('Ranked venues:', ranked_venues.shape[0])
print('Price range venues:', price_range_venues.shape[0])
venue_df = pd.merge(ranked_venues,price_range_venues,on='Venue_ID', how='inner')
print('Combined venues:', venue_df.shape[0])

Ranked venues: 242
Price range venues: 240
Combined venues: 240


In [ ]:
venue_df.head(5)

,Venue_Rank,Venue_ID,Venue_Name,Venue_Postal_Code,Venue_Latitude,Venue_Longitude,Venue_Category,Venue_Price_Category
0,0,5a32fb62c530935f37812611,Eureka!,89101,36.168976,-115.139580,American Restaurant,2
1,1,51cde2b08bbd23404bdc1798,Pizza Rock,89101,36.171707,-115.142343,Pizza Place,2
2,2,552ae36a498e9b3b1e232a6e,VegeNation,89101,36.167398,-115.139421,Vegetarian / Vegan Restaurant,2
3,3,539a4129498e2eba5804ba4a,Carson Kitchen,89101,36.167884,-115.140664,Gastropub,3
4,4,510a01d5e4b07b50d4c6a4ae,Joe Vicari's Andiamo Steakhouse,89101,36.169942,-115.142740,Steakhouse,4


In [ ]:
# Joining the venue dataframe to the postal code dataframe to also include that
analysis_df = pd.merge(venue_df, location_df, left_on='Venue_Postal_Code', right_on='Postal_Code')
print('Number of venues for analysis:', analysis_df.shape[0])
analysis_df.head(10)

Number of venues for analysis: 235


,Venue_Rank,Venue_ID,Venue_Name,Venue_Postal_Code,Venue_Latitude,Venue_Longitude,Venue_Category,Venue_Price_Category,Postal_Code,Place_Name,State_Name,State_Code,Country_Code,PC_Latitude,PC_Longitude
0,0,5a32fb62c530935f37812611,Eureka!,89101,36.168976,-115.139580,American Restaurant,2,89101,Las Vegas,Nevada,NV,US,36.17193,-115.14001
1,1,51cde2b08bbd23404bdc1798,Pizza Rock,89101,36.171707,-115.142343,Pizza Place,2,89101,Las Vegas,Nevada,NV,US,36.17193,-115.14001
2,2,552ae36a498e9b3b1e232a6e,VegeNation,89101,36.167398,-115.139421,Vegetarian / Vegan Restaurant,2,89101,Las Vegas,Nevada,NV,US,36.17193,-115.14001
3,3,539a4129498e2eba5804ba4a,Carson Kitchen,89101,36.167884,-115.140664,Gastropub,3,89101,Las Vegas,Nevada,NV,US,36.17193,-115.14001
4,4,510a01d5e4b07b50d4c6a4ae,Joe Vicari's Andiamo Steakhouse,89101,36.169942,-115.142740,Steakhouse,4,89101,Las Vegas,Nevada,NV,US,36.17193,-115.14001
5,5,4ea5ca369adf05b8d0006a20,Le Thai,89101,36.168839,-115.139921,Thai Restaurant,2,89101,Las Vegas,Nevada,NV,US,36.17193,-115.14001
6,6,503cec78e4b0f39ae12141db,eat.,89101,36.166927,-115.139055,Breakfast Spot,1,89101,Las Vegas,Nevada,NV,US,36.17193,-115.14001
7,7,4fa59a6fe4b0bbcd4a17c02b,Pop Up Pizza,89101,36.171171,-115.147438,Pizza Place,1,89101,Las Vegas,Nevada,NV,US,36.17193,-115.14001
8,9,4e5d456b88771beefa96b52d,Hash House A Go Go,89101,36.171612,-115.146466,Breakfast Spot,2,89101,Las Vegas,Nevada,NV,US,36.17193,-115.14001
9,11,4b4957b9f964a520e16d26e3,Hugo's Cellar,89101,36.170524,-115.143849,Steakhouse,3,89101,Las Vegas,Nevada,NV,US,36.17193,-115.14001


In [ ]:
analysis_df.dtypes

Venue_Rank                int64
Venue_ID                 object
Venue_Name               object
Venue_Postal_Code        object
Venue_Latitude          float64
Venue_Longitude         float64
Venue_Category           object
Venue_Price_Category      int64
Postal_Code              object
Place_Name               object
State_Name               object
State_Code               object
Country_Code             object
PC_Latitude             float64
PC_Longitude            float64
dtype: object

To get a feel for the locations we are analyzing, let's plot them into a map.

In [ ]:
import branca.colormap

# create map
map_price_range = folium.Map(location=[latitude, longitude], 
                          tiles='OpenStreetMap',
                          zoom_start=12)

# set color scheme for the clusters
price_cat = 4 # 4 price categories
x = np.arange(price_cat)
ys = [i + x + (i*x)**2 for i in range(price_cat)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, v_name, p_cat in zip(analysis_df['Venue_Latitude'], analysis_df['Venue_Longitude'], analysis_df['Venue_Name'], analysis_df['Venue_Price_Category']):
    label = folium.Popup(str(v_name) + ' Price Category ' + str(p_cat), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=4,
        popup=label,
        color=rainbow[p_cat-1],
        fill=True,
        fill_color=rainbow[p_cat-1],
        fill_opacity=0.9).add_to(map_price_range)

colormap = branca.colormap.linear.YlOrRd_09.scale(0, 5)
colormap = colormap.to_step(index=[1, 2, 3, 4, 5])
colormap.caption = 'Price range legend'
colormap.add_to(map_price_range)
       
map_price_range

Given the above, we can use K-Means clustering with the popularity/ranking as a feature to investigate which areas / groups are the most relevant for business opportunities.

Before proceeding, we will modify the ranking of the analysis dataframe to start at "1" rather than "0" as this will improve the overall workflow.

In [ ]:
# Ranking venues with 1-indexation for later on
analysis_df['Venue_Rank'] = list(range(1,analysis_df.shape[0]+1))

# Changing data types of Postal Codes, Price Categories, into categorial data types
analysis_df['Venue_Postal_Code'] = analysis_df['Venue_Postal_Code'].astype('category')
analysis_df['Venue_Category'] = analysis_df['Venue_Category'].astype('category')
analysis_df['Venue_Price_Category'] = analysis_df['Venue_Price_Category'].astype('category')

## Predicting business opportunities

#### Clustering neighborhoods based on popularity

We will investigate which neighborhoods have the most venues according to price category.

**Input:** Postal Code, Ranking  
**Output:** Venue groups according to rating

In [ ]:
# break down the price categories into one hot encoding:
pricecat_onehot = pd.get_dummies(analysis_df[['Venue_Price_Category']], prefix='', prefix_sep='')

# add the postal code into the one hot dataframe
pricecat_onehot = pd.concat([analysis_df['Venue_Postal_Code'], pricecat_onehot], axis=1)

In [ ]:
# grouping the postal areas according to the mean number of each price category
pricecat_grouped = pricecat_onehot.groupby('Venue_Postal_Code').mean().reset_index()
pricecat_grouped.head(3)

,Venue_Postal_Code,1,2,3,4
0,89014,0.5,0.375,0.125,0.0
1,89030,0.5,0.500,0.000,0.0
2,89031,0.0,1.000,0.000,0.0


In [ ]:
# creating a function to return each price category according to popularity
def return_sorted_rows(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
len(analysis_df['Venue_Price_Category'].unique())

4

In [ ]:
# create new dataframe to store the ranked results
pricecat_ranked = pd.DataFrame(columns=pricecat_grouped.columns)
pricecat_ranked['Venue_Postal_Code'] = pricecat_grouped['Venue_Postal_Code']

for row in np.arange(pricecat_grouped.shape[0]):
  pricecat_ranked.iloc[row, 1:] = return_sorted_rows(pricecat_grouped.iloc[row, :], len(analysis_df['Venue_Price_Category'].unique()))

pricecat_ranked.head(5)

,Venue_Postal_Code,1,2,3,4
0,89014,1,2,3,4
1,89030,2,1,4,3
2,89031,2,4,3,1
3,89032,2,4,3,1
4,89081,1,4,3,2


Clustering venues and postal codes according to price categories.

In [ ]:
# defining number of clusters
klusters = 5

# keep only the relevant features
pricecat_clustering = pricecat_grouped.drop('Venue_Postal_Code', 1)

# run K-means clustering
kmeans = KMeans(n_clusters=klusters, random_state=0).fit(pricecat_clustering)

# have a look at the cluster categories for each row
kmeans.labels_

array([2, 2, 1, 1, 3, 2, 4, 2, 2, 0, 1, 1, 2, 4, 3, 1, 2, 2, 4, 1, 1, 2,
       2, 0, 0, 3, 4, 2, 1, 1, 0, 4], dtype=int32)

In [ ]:
# add clusters back into dataframe
pricecat_ranked.insert(0, 'Price_Cluster', kmeans.labels_)
pricecat_ranked.head(2)

,Price_Cluster,Venue_Postal_Code,1,2,3,4
0,2,89014,1,2,3,4
1,2,89030,2,1,4,3


In [ ]:
pricecat_ranked.head(2)

,Price_Cluster,Venue_Postal_Code,1,2,3,4
0,2,89014,1,2,3,4
1,2,89030,2,1,4,3


In [ ]:
postal_codes_df.head(3)

,Postal_Code,Place_Name,State_Name,State_Code,Country_Code,PC_Latitude,PC_Longitude
65,89014,Henderson,Nevada,NV,US,36.056435,-115.077968
32,89030,North Las Vegas,Nevada,NV,US,36.211457,-115.124147
53,89031,North Las Vegas,Nevada,NV,US,36.258888,-115.171801


In [ ]:
analysis_df[['Postal_Code', 'Place_Name', 'PC_Latitude', 'PC_Longitude']].drop_duplicates().head(3)

,Postal_Code,Place_Name,PC_Latitude,PC_Longitude
0,89101,Las Vegas,36.171930,-115.140010
10,89102,Las Vegas,36.143303,-115.200351
30,89106,Las Vegas,36.184673,-115.161703


In [ ]:
# Creating merged dataframe to contain all the relevant data
pricecat_merged = analysis_df[['Postal_Code', 'Place_Name', 'PC_Latitude', 'PC_Longitude']].drop_duplicates()
pricecat_merged = pricecat_merged.join(pricecat_ranked.set_index('Venue_Postal_Code'), on='Postal_Code', how='right')
# casting the various price categories as integers for numerical operations
pricecat_merged[['1','2', '3', '4']] = pricecat_merged[['1','2', '3', '4']].astype('int32')
pricecat_merged.sort_values(by='Price_Cluster').head(10)

,Postal_Code,Place_Name,PC_Latitude,PC_Longitude,Price_Cluster,1,2,3,4
178,89130,Las Vegas,36.247137,-115.221032,0,1,3,2,4
209,89128,Las Vegas,36.196823,-115.264370,0,3,1,4,2
149,89158,Las Vegas,36.174971,-115.137223,0,3,4,1,2
31,89104,Las Vegas,36.151970,-115.109195,0,1,3,4,2
143,89115,Las Vegas,36.215818,-115.067062,1,2,4,3,1
194,89156,Las Vegas,36.203430,-115.036376,1,2,4,3,1
193,89031,North Las Vegas,36.258888,-115.171801,1,2,4,3,1
146,89032,North Las Vegas,36.217968,-115.170919,1,2,4,3,1
229,89147,Las Vegas,36.112795,-115.280099,1,2,4,3,1
220,89120,Las Vegas,36.091423,-115.088485,1,2,4,3,1


In [ ]:
pricecat_merged.groupby('Price_Cluster').agg({'1': ['count', 'mean'], '2': 'mean', '3':'mean', '4':'mean'})
#agg({'B': ['min', 'max'], 'C': 'sum'})

1                   2         3         4
              count      mean      mean      mean      mean
Price_Cluster                                              
0                 4  2.000000  2.750000  2.750000  2.500000
1                 9  2.000000  4.000000  3.000000  1.000000
2                11  1.909091  1.090909  3.727273  3.272727
3                 3  1.000000  4.000000  3.000000  2.000000
4                 5  2.200000  2.600000  1.800000  3.400000

From the above, we can see there are actually minor differences between the price categories per area; with the exception of cluster 4, the other clusters have a similar price range (1.9-2.2) as the most popular type.

Creating a map of the clusters.

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(klusters)
ys = [i + x + (i*x)**2 for i in range(klusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pricecat_merged['PC_Latitude'], pricecat_merged['PC_Longitude'], pricecat_merged['Place_Name'], pricecat_merged['Price_Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
#map_clusters

#### Clustering venues based on popularity

Steps: 
1. Assign venue popularity according to groups (1-5)
2. Perform one-hot encoding to identify the groups of venue popularity per postal code (can also be done per price category)

In [ ]:
# Grouping the venue ratings into categories
pop_groups = 5 # making 5 groups
pop_labels = ['Very high', 'High', 'Medium', 'Low', 'Very low']
analysis_df['Venue_Popularity'] = pd.cut(analysis_df['Venue_Rank'],
                                             bins = pop_groups,
                                             labels = pop_labels,
                                             precision = 0,
                                             ordered = False)
analysis_df['Venue_Popularity'].value_counts()

Very low     47
Low          47
Medium       47
High         47
Very high    47
Name: Venue_Popularity, dtype: int64

In [ ]:
# convervting to one hot encoding using dummies
pop_onehot = pd.get_dummies(analysis_df[['Venue_Popularity']], prefix='', prefix_sep='')
pop_onehot = pd.concat([analysis_df['Venue_Postal_Code'], pop_onehot], axis=1)

In [ ]:
pop_grouped = pop_onehot.groupby('Venue_Postal_Code').mean().reset_index()
pop_grouped.head(3)

,Venue_Postal_Code,Very high,High,Medium,Low,Very low
0,89014,0.0,0.0,0.0,0.0,1.0
1,89030,0.0,0.0,1.0,0.0,0.0
2,89031,0.0,0.0,0.0,0.0,1.0


In [ ]:
# creating a function to return each price category according to popularity
def return_sorted_rows(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Clustering venues and postal codes according to price categories.

In [ ]:
# defining number of clusters
klusters = 5

# keep only the relevant features
popularity_cluster = pop_grouped.drop('Venue_Postal_Code', 1)

# run K-means clustering
kmeans = KMeans(n_clusters=klusters, random_state=0).fit(popularity_cluster)

# have a look at the cluster categories for each row
kmeans.labels_

array([2, 3, 2, 0, 2, 2, 1, 1, 0, 1, 1, 0, 0, 4, 0, 0, 2, 2, 3, 2, 3, 2,
       2, 2, 0, 2, 2, 0, 2, 2, 0, 3], dtype=int32)

In [ ]:
# add clusters back into dataframe
pop_grouped.insert(1, 'Popularity_Cluster', kmeans.labels_)
pop_grouped.head(10)

,Venue_Postal_Code,Popularity_Cluster,Very high,High,Medium,Low,Very low
0,89014,2,0.0,0.0,0.0,0.0,1.0
1,89030,3,0.0,0.0,1.0,0.0,0.0
2,89031,2,0.0,0.0,0.0,0.0,1.0
3,89032,0,0.0,0.0,0.0,1.0,0.0
4,89081,2,0.0,0.0,0.0,0.0,1.0
5,89084,2,0.0,0.0,0.0,0.0,1.0
6,89101,1,1.0,0.0,0.0,0.0,0.0
7,89102,1,1.0,0.0,0.0,0.0,0.0
8,89103,0,0.0,0.0,0.0,1.0,0.0
9,89104,1,1.0,0.0,0.0,0.0,0.0


In [ ]:
pop_grouped.sort_values(by='Popularity_Cluster')

,Venue_Postal_Code,Popularity_Cluster,Very high,High,Medium,Low,Very low
15,89115,0,0.000000,0.000000,0.000000,1.000000,0.000000
24,89130,0,0.000000,0.000000,0.000000,1.000000,0.000000
30,89158,0,0.000000,0.000000,0.000000,1.000000,0.000000
3,89032,0,0.000000,0.000000,0.000000,1.000000,0.000000
27,89146,0,0.000000,0.000000,0.000000,1.000000,0.000000
14,89110,0,0.000000,0.000000,0.000000,1.000000,0.000000
12,89108,0,0.000000,0.000000,0.000000,1.000000,0.000000
11,89107,0,0.000000,0.000000,0.000000,1.000000,0.000000
8,89103,0,0.000000,0.000000,0.000000,1.000000,0.000000
10,89106,1,1.000000,0.000000,0.000000,0.000000,0.000000


So what have we done above?
- We have clustered the various areas according to their popularity.

What does this tell us?
1. There are some patters between the popularity of the various neighborhoods; but that's nothing surprising.

#### Full K-means: Include all features in analysis

**Task: To include a full k-Means analysis for the project**
1. Include all features into a dataframe, one-hot enocde the data
2. We actually want to cluster according to Popularity. This is not really k-means (unsupervised) learning, but supervised learning. However, let's try k-Means first.
3. We will do this by postal codes
4. In the final features-treatment, we could do some tricks. Only include the topmost popularity categories for instance.

In [ ]:
# Grouping the venue ratings into categories
pop_groups = 5 # making 5 groups
pop_labels = ['Very high', 'High', 'Medium', 'Low', 'Very low']
analysis_df['Venue_Popularity'] = pd.cut(analysis_df['Venue_Rank'],
                                             bins = pop_groups,
                                             labels = pop_labels,
                                             precision = 0,
                                             ordered = False)
analysis_df['Venue_Popularity'].value_counts()

Very low     47
Low          47
Medium       47
High         47
Very high    47
Name: Venue_Popularity, dtype: int64

In [ ]:
# Converting Venue Popularity to one hot encoding using dummies
pop_onehot = pd.get_dummies(analysis_df[['Venue_Popularity']], prefix='', prefix_sep='')
pop_onehot = pd.concat([analysis_df['Venue_Postal_Code'], pop_onehot], axis=1)

# Grouping the postal codes according to the mean number of each popularity category
pop_grouped = pop_onehot.groupby('Venue_Postal_Code').mean().reset_index()

# Renaming the columns for legibility:
colnames = pop_grouped.columns.values
colnames = np.append(colnames[0], 'Popularity ' + np.array(colnames[1:]))
pop_grouped.columns = colnames

pop_grouped.head(3)

,Venue_Postal_Code,Popularity Very high,Popularity High,Popularity Medium,Popularity Low,Popularity Very low
0,89014,0.0,0.0,0.0,0.0,1.0
1,89030,0.0,0.0,1.0,0.0,0.0
2,89031,0.0,0.0,0.0,0.0,1.0


In [ ]:
# Convert Venue Rating to one hot encoding using dummies
pricecat_onehot = pd.get_dummies(analysis_df[['Venue_Price_Category']], prefix='', prefix_sep='')

# add the postal code into the one hot dataframe
pricecat_onehot = pd.concat([analysis_df['Venue_Postal_Code'], pricecat_onehot], axis=1)

# grouping the postal areas according to the mean number of each price category
pricecat_grouped = pricecat_onehot.groupby('Venue_Postal_Code').mean().reset_index()

# Renaming price category dummy names for legibility
colnames = pricecat_grouped.columns.values
colnames = np.append(colnames[0], 'Price Category ' + np.array(colnames[1:]))
pricecat_grouped.columns = colnames

pricecat_grouped.head(3)

,Venue_Postal_Code,Price Category 1,Price Category 2,Price Category 3,Price Category 4
0,89014,0.5,0.375,0.125,0.0
1,89030,0.5,0.500,0.000,0.0
2,89031,0.0,1.000,0.000,0.0


In [ ]:
analysis_df.head(2)

,Venue_Rank,Venue_ID,Venue_Name,Venue_Postal_Code,Venue_Latitude,Venue_Longitude,Venue_Category,Venue_Price_Category,Postal_Code,Place_Name,State_Name,State_Code,Country_Code,PC_Latitude,PC_Longitude,Venue_Popularity
0,1,5a32fb62c530935f37812611,Eureka!,89101,36.168976,-115.139580,American Restaurant,2,89101,Las Vegas,Nevada,NV,US,36.17193,-115.14001,Very high
1,2,51cde2b08bbd23404bdc1798,Pizza Rock,89101,36.171707,-115.142343,Pizza Place,2,89101,Las Vegas,Nevada,NV,US,36.17193,-115.14001,Very high


In [ ]:
# Convert Venue Category to one hot encoding using dummies
ven_cat_onehot = pd.get_dummies(analysis_df[['Venue_Category']], prefix='', prefix_sep='')

# add the postal code into the one hot dataframe
ven_cat_onehot = pd.concat([analysis_df['Venue_Postal_Code'], ven_cat_onehot], axis=1)

# grouping the postal areas according to the mean number of each price category
ven_cat_grouped = ven_cat_onehot.groupby('Venue_Postal_Code').mean().reset_index()

# Renaming price category dummy names for legibility
colnames = ven_cat_grouped.columns.values
colnames = np.append(colnames[0], 'Category: ' + np.array(colnames[1:]))
ven_cat_grouped.columns = colnames

ven_cat_grouped.head(3)

,Venue_Postal_Code,Category: American Restaurant,Category: Andhra Restaurant,Category: Arepa Restaurant,Category: Argentinian Restaurant,Category: Asian Restaurant,Category: BBQ Joint,Category: Bagel Shop,Category: Bakery,Category: Brazilian Restaurant,Category: Breakfast Spot,Category: Buffet,Category: Burger Joint,Category: Café,Category: Cajun / Creole Restaurant,Category: Caribbean Restaurant,Category: Chinese Restaurant,Category: Deli / Bodega,Category: Diner,Category: Eastern European Restaurant,Category: Fast Food Restaurant,Category: French Restaurant,Category: Fried Chicken Joint,Category: Gastropub,Category: Greek Restaurant,Category: Hawaiian Restaurant,Category: Hot Dog Joint,Category: Indian Restaurant,Category: Irish Pub,Category: Italian Restaurant,Category: Japanese Restaurant,Category: Korean Restaurant,Category: Latin American Restaurant,Category: Mexican Restaurant,Category: Middle Eastern Restaurant,Category: New American Restaurant,Category: Noodle House,Category: Pizza Place,Category: Restaurant,Category: Sandwich Place,Category: Seafood Restaurant,Category: Snack Place,Category: Southern / Soul Food Restaurant,Category: Spanish Restaurant,Category: Steakhouse,Category: Sushi Restaurant,Category: Taco Place,Category: Tapas Restaurant,Category: Thai Restaurant,Category: Theme Restaurant,Category: Vegetarian / Vegan Restaurant,Category: Vietnamese Restaurant,Category: Wings Joint
0,89014,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.125,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.00,0.0,0.0,0.0,0.0,0.125,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,89030,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.250,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.75,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,89031,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
# Joining the dummies dataframes together
areas_grouped = pd.merge(pop_grouped, pricecat_grouped, on='Venue_Postal_Code')
areas_grouped = pd.merge(areas_grouped, ven_cat_grouped, on='Venue_Postal_Code')
areas_grouped

,Venue_Postal_Code,Popularity Very high,Popularity High,Popularity Medium,Popularity Low,Popularity Very low,Price Category 1,Price Category 2,Price Category 3,Price Category 4,Category: American Restaurant,Category: Andhra Restaurant,Category: Arepa Restaurant,Category: Argentinian Restaurant,Category: Asian Restaurant,Category: BBQ Joint,Category: Bagel Shop,Category: Bakery,Category: Brazilian Restaurant,Category: Breakfast Spot,Category: Buffet,Category: Burger Joint,Category: Café,Category: Cajun / Creole Restaurant,Category: Caribbean Restaurant,Category: Chinese Restaurant,Category: Deli / Bodega,Category: Diner,Category: Eastern European Restaurant,Category: Fast Food Restaurant,Category: French Restaurant,Category: Fried Chicken Joint,Category: Gastropub,Category: Greek Restaurant,Category: Hawaiian Restaurant,Category: Hot Dog Joint,Category: Indian Restaurant,Category: Irish Pub,Category: Italian Restaurant,Category: Japanese Restaurant,Category: Korean Restaurant,Category: Latin American Restaurant,Category: Mexican Restaurant,Category: Middle Eastern Restaurant,Category: New American Restaurant,Category: Noodle House,Category: Pizza Place,Category: Restaurant,Category: Sandwich Place,Category: Seafood Restaurant,Category: Snack Place,Category: Southern / Soul Food Restaurant,Category: Spanish Restaurant,Category: Steakhouse,Category: Sushi Restaurant,Category: Taco Place,Category: Tapas Restaurant,Category: Thai Restaurant,Category: Theme Restaurant,Category: Vegetarian / Vegan Restaurant,Category: Vietnamese Restaurant,Category: Wings Joint
0,89014,0.000000,0.000000,0.000000,0.000000,1.000000,0.500000,0.375000,0.125000,0.000000,0.000000,0.125000,0.00,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.125000,0.0,0.00,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.125,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,89030,0.000000,0.000000,1.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.250000,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,89031,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
3,89032,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333
4,89081,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.000000,0.000000,0.

At this stage, we will later do the trick: To remove any non-popular venues. That way, the K-means analysis will be an analysis on the venues that are popular! But we should probably also include the postal codes - and venue categories?

Clustering venues and postal codes according to price categories.

In [ ]:
# defining number of clusters
klusters = 5

# keep only the relevant features; i.e. dropping the postal code
areas_cluster = areas_grouped.drop('Venue_Postal_Code', 1)

# run K-means clustering
kmeans = KMeans(n_clusters=klusters, random_state=0).fit(areas_cluster)

# have a look at the cluster categories for each row
kmeans.labels_

array([4, 3, 2, 0, 4, 2, 1, 1, 0, 1, 1, 0, 0, 3, 0, 0, 2, 4, 3, 2, 3, 4,
       2, 4, 0, 4, 2, 0, 2, 2, 0, 3], dtype=int32)

In [ ]:
# add clusters back into dataframe
areas_grouped.insert(1, 'Category_Cluster', kmeans.labels_)
areas_grouped.head(10)

,Venue_Postal_Code,Category_Cluster,Popularity Very high,Popularity High,Popularity Medium,Popularity Low,Popularity Very low,Price Category 1,Price Category 2,Price Category 3,Price Category 4,Category: American Restaurant,Category: Andhra Restaurant,Category: Arepa Restaurant,Category: Argentinian Restaurant,Category: Asian Restaurant,Category: BBQ Joint,Category: Bagel Shop,Category: Bakery,Category: Brazilian Restaurant,Category: Breakfast Spot,Category: Buffet,Category: Burger Joint,Category: Café,Category: Cajun / Creole Restaurant,Category: Caribbean Restaurant,Category: Chinese Restaurant,Category: Deli / Bodega,Category: Diner,Category: Eastern European Restaurant,Category: Fast Food Restaurant,Category: French Restaurant,Category: Fried Chicken Joint,Category: Gastropub,Category: Greek Restaurant,Category: Hawaiian Restaurant,Category: Hot Dog Joint,Category: Indian Restaurant,Category: Irish Pub,Category: Italian Restaurant,Category: Japanese Restaurant,Category: Korean Restaurant,Category: Latin American Restaurant,Category: Mexican Restaurant,Category: Middle Eastern Restaurant,Category: New American Restaurant,Category: Noodle House,Category: Pizza Place,Category: Restaurant,Category: Sandwich Place,Category: Seafood Restaurant,Category: Snack Place,Category: Southern / Soul Food Restaurant,Category: Spanish Restaurant,Category: Steakhouse,Category: Sushi Restaurant,Category: Taco Place,Category: Tapas Restaurant,Category: Thai Restaurant,Category: Theme Restaurant,Category: Vegetarian / Vegan Restaurant,Category: Vietnamese Restaurant,Category: Wings Joint
0,89014,4,0.0,0.0,0.0,0.0,1.0,0.500000,0.375,0.125,0.000000,0.00,0.125000,0.00,0.0,0.000000,0.0,0.0,0.125,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.125,0.0,0.000000,0.000000,0.0,0.125000,0.0,0.0,0.125,0.0,0.0,0.000000,0.000000,0.00,0.125,0.0,0.000000,0.0,0.00,0.000000,0.0,0.125,0.125,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.00,0.000000
1,89030,3,0.0,0.0,1.0,0.0,0.0,0.500000,0.500,0.000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.000,0.0,0.000000,0.000000,0.0,0.250000,0.0,0.0,0.000,0.0,0.0,0.000000,0.000000,0.00,0.000,0.0,0.750000,0.0,0.00,0.000000,0.0,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.00,0.000000
2,89031,2,0.0,0.0,0.0,0.0,1.0,0.000000,1.000,0.000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000000,0.000000,0.00,0.000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,1.000000,0.0,0.0,0.00,0.000000
3,89032,0,0.0,0.0,0.0,1.0,0.0,0.000000,1.000,0.000,0.000000,0.00,0.000000,0.00,0.0,0.333333,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000000,0.000000,0.00,0.000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.333333,0.00,0.0,0.000000,0.0,0.0,0.00,0.333333
4,89081,4,0.0,0.0,0.0,0.0,1.0,1.000000,0.000,0.000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.0,0.0,0.000,0.0,0.000000,0.0,1.0,0.0,0.00,0.0,0.0,0.000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000000,0.000000,0.00,0.000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.00,0.000000
5,89084,2,0.0,0.0,0.0,0.0,1.0,0.500000,0.500,0.000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000000,0.000000,0.00,0.000,0.0,0.000000,0.0,0.00,0.000000,0.5,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.500000,0.00,0.0,0.000000,0.0,0.0,0.00,0.000000
6,89101,1,1.0,0.0,0.0,0.0,0.0,0.200000,0.500,0.200,0.100000,0.10,0.000000,0.00,0.0,0.000000,0.0,0.0,0.000,0.0,0.200000,0.0,0.0,0.0,0.00,0.0,0.0,0.000,0.0,0.000000,0.000000,0.0,0.000000,0.1,0.0,0.000,0.0,0.0,0.000000,0.000000,0.00,0.000,0.0,0.000000,0.0,0.00,0.

In [ ]:
areas_grouped.sort_values(by='Category_Cluster')

,Venue_Postal_Code,Category_Cluster,Popularity Very high,Popularity High,Popularity Medium,Popularity Low,Popularity Very low,Price Category 1,Price Category 2,Price Category 3,Price Category 4,Category: American Restaurant,Category: Andhra Restaurant,Category: Arepa Restaurant,Category: Argentinian Restaurant,Category: Asian Restaurant,Category: BBQ Joint,Category: Bagel Shop,Category: Bakery,Category: Brazilian Restaurant,Category: Breakfast Spot,Category: Buffet,Category: Burger Joint,Category: Café,Category: Cajun / Creole Restaurant,Category: Caribbean Restaurant,Category: Chinese Restaurant,Category: Deli / Bodega,Category: Diner,Category: Eastern European Restaurant,Category: Fast Food Restaurant,Category: French Restaurant,Category: Fried Chicken Joint,Category: Gastropub,Category: Greek Restaurant,Category: Hawaiian Restaurant,Category: Hot Dog Joint,Category: Indian Restaurant,Category: Irish Pub,Category: Italian Restaurant,Category: Japanese Restaurant,Category: Korean Restaurant,Category: Latin American Restaurant,Category: Mexican Restaurant,Category: Middle Eastern Restaurant,Category: New American Restaurant,Category: Noodle House,Category: Pizza Place,Category: Restaurant,Category: Sandwich Place,Category: Seafood Restaurant,Category: Snack Place,Category: Southern / Soul Food Restaurant,Category: Spanish Restaurant,Category: Steakhouse,Category: Sushi Restaurant,Category: Taco Place,Category: Tapas Restaurant,Category: Thai Restaurant,Category: Theme Restaurant,Category: Vegetarian / Vegan Restaurant,Category: Vietnamese Restaurant,Category: Wings Joint
15,89115,0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24,89130,0,0.000000,0.000000,0.000000,1.000000,0.000000,0.600000,0.200000,0.200000,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.200000,0.000000,0.000000,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.200000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
30,89158,0,0.000000,0.000000,0.000000,1.000000,0.000000,0.250000,0.000000,0.500000,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,89032,0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333
14,89110,0,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.500000,0.000000,0.000000,0.0,

In [ ]:
# Grouping on the given clusters to view their relevant details
areas_grouped.groupby('Category_Cluster').mean()

,Popularity Very high,Popularity High,Popularity Medium,Popularity Low,Popularity Very low,Price Category 1,Price Category 2,Price Category 3,Price Category 4,Category: American Restaurant,Category: Andhra Restaurant,Category: Arepa Restaurant,Category: Argentinian Restaurant,Category: Asian Restaurant,Category: BBQ Joint,Category: Bagel Shop,Category: Bakery,Category: Brazilian Restaurant,Category: Breakfast Spot,Category: Buffet,Category: Burger Joint,Category: Café,Category: Cajun / Creole Restaurant,Category: Caribbean Restaurant,Category: Chinese Restaurant,Category: Deli / Bodega,Category: Diner,Category: Eastern European Restaurant,Category: Fast Food Restaurant,Category: French Restaurant,Category: Fried Chicken Joint,Category: Gastropub,Category: Greek Restaurant,Category: Hawaiian Restaurant,Category: Hot Dog Joint,Category: Indian Restaurant,Category: Irish Pub,Category: Italian Restaurant,Category: Japanese Restaurant,Category: Korean Restaurant,Category: Latin American Restaurant,Category: Mexican Restaurant,Category: Middle Eastern Restaurant,Category: New American Restaurant,Category: Noodle House,Category: Pizza Place,Category: Restaurant,Category: Sandwich Place,Category: Seafood Restaurant,Category: Snack Place,Category: Southern / Soul Food Restaurant,Category: Spanish Restaurant,Category: Steakhouse,Category: Sushi Restaurant,Category: Taco Place,Category: Tapas Restaurant,Category: Thai Restaurant,Category: Theme Restaurant,Category: Vegetarian / Vegan Restaurant,Category: Vietnamese Restaurant,Category: Wings Joint
Category_Cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.000000,0.000000,1.000000,0.000000,0.341595,0.535043,0.086325,0.037037,0.030769,0.009259,0.0000,0.008547,0.037037,0.022222,0.000000,0.000000,0.000000,0.017806,0.000000,0.008547,0.147436,0.0000,0.000000,0.000000,0.000000,0.000000,0.009259,0.142593,0.000000,0.009259,0.000,0.000000,0.000000,0.000000,0.000000,0.009259,0.009259,0.025641,0.008547,0.000000,0.037037,0.00000,0.000000,0.027066,0.022222,0.000000,0.119658,0.064815,0.000000,0.000000,0.000000,0.022222,0.045584,0.055556,0.000000,0.064815,0.000000,0.000000,0.008547,0.037037
1,1.000000,0.000000,0.000000,0.000000,0.000000,0.312500,0.512500,0.150000,0.025000,0.350000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.0125,0.000000,0.000000,0.000000,0.000000,0.000000,0.087500,0.000000,0.012500,0.025,0.000000,0.000000,0.000000,0.000000,0.000000,0.012500,0.012500,0.000000,0.000000,0.012500,0.00000,0.012500,0.000000,0.050000,0.000000,0.012500,0.012500,0.000000,0.000000,0.000000,0.050000,0.037500,0.062500,0.000000,0.037500,0.000000,0.025000,0.012500,0.000000
2,0.000000,0.000000,0.000000,0.053571,0.946429,0.172024,0.760516,0.053571,0.013889,0.035714,0.025000,0.0000,0.000000,0.000000,0.000000,0.017857,0.000000,0.000000,0.107639,0.000000,0.025000,0.031746,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031746,0.031250,0.000000,0.000000,0.163889,0.03125,0.000000,0.000000,0.090278,0.000000,0.013889,0.000000,0.000000,0.000000,0.000000,0.056746,0.170139,0.000000,0.000000,0.125000,0.000000,0.017857,0.000000,0.000000
3,0.030769,0.120513,0.835385,0.013333,0.000000,0.246374,0.543150,0.143297,0.067179,0.028718,0.000000,0.0000,0.000000,0.005128,0.002564,0.000000,0.002564,0.041905,0.071795,0.005128,0.023590,0.002564,0.0000,0.013333,0.005128,0.000000,0.002564,0.000000,0.015897,0.020513,0.065897,0.000,0.000000,0.000000,0.002564,0.013333,0.015897,0.049744,0.018462,0.028571,0.002564,0.232564,0.00000,0.015897,0.000000,0.005128,0.000000,0.031136,0.005128,0.005128,0.002564,0.002564,0.090476,0.090256,0.031136,0.028571,0.002564,0.002564,0.013333,0.000000,0.002564
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.666667,0.229167,0.104167,0.000000,0.000000,0.020833,0.0000,0.000000,0.000000,0.000000,0.000000,0.104167,0.000000,0.333333,0.000000,0.16666

The above gives us very useful feedback. The main findings are:
- Category cluster 1 contain the most popular ("Popularity Very High") venues. If we are to look for business opportunities, we should look at the features of the venues in this category.
- Category cluster 3 contain the venues that are of a medium popularity (83.5% of its venues are in this popularity), though some venues also have higher popularity.
- The category clusters 0, 2, and 4 should certainly be avoided. These have a low- to very low popularity.

Let's look at the features of the popular venues - what we should aim at when creating a new business.

In [ ]:
# At this stage, we should extract information / details about what features and areas make a vene popular
pop_venues = areas_grouped.loc[(areas_grouped['Category_Cluster'] == 1) | (areas_grouped['Category_Cluster'] == 3)]
pop_venues.sort_values(by=['Category_Cluster'])

,Venue_Postal_Code,Category_Cluster,Popularity Very high,Popularity High,Popularity Medium,Popularity Low,Popularity Very low,Price Category 1,Price Category 2,Price Category 3,Price Category 4
6,89101,1,1.000000,0.000000,0.000000,0.000000,0.0,0.200000,0.500000,0.200000,0.100000
7,89102,1,1.000000,0.000000,0.000000,0.000000,0.0,0.300000,0.550000,0.150000,0.000000
9,89104,1,1.000000,0.000000,0.000000,0.000000,0.0,0.750000,0.000000,0.250000,0.000000
10,89106,1,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000
1,89030,3,0.000000,0.000000,1.000000,0.000000,0.0,0.500000,0.500000,0.000000,0.000000
13,89109,3,0.153846,0.602564,0.243590,0.000000,0.0,0.179487,0.320513,0.230769,0.269231
18,89119,3,0.000000,0.000000,0.933333,0.066667,0.0,0.266667,0.466667,0.200000,0.066667
20,89121,3,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000
31,89169,3,0.000000,0.000000,1.000000,0.000000,0.0,0.285714,0.428571,0.285714,0.000000


In [ ]:
business_opp = areas_grouped.loc[areas_grouped['Category_Cluster'] == 1].sort_values(by=['Venue_Postal_Code'])
business_opp

,Venue_Postal_Code,Category_Cluster,Popularity Very high,Popularity High,Popularity Medium,Popularity Low,Popularity Very low,Price Category 1,Price Category 2,Price Category 3,Price Category 4
6,89101,1,1.0,0.0,0.0,0.0,0.0,0.20,0.50,0.20,0.1
7,89102,1,1.0,0.0,0.0,0.0,0.0,0.30,0.55,0.15,0.0
9,89104,1,1.0,0.0,0.0,0.0,0.0,0.75,0.00,0.25,0.0
10,89106,1,1.0,0.0,0.0,0.0,0.0,0.00,1.00,0.00,0.0


What can we conclude from the above?

In [ ]:
# Make a count of the price categories per postal code
#analysis_df.loc[analysis_df['Venue_Postal_Code'].isin(business_opp['Venue_Postal_Code'])]

In [ ]:
# Make a graph with the price categories of each price category per postal code

So what have we done above?
- We have clustered the various areas according to their popularity.

What does this tell us?
1. There are some patters between the popularity of the various neighborhoods; but that's nothing surprising.

In [ ]:
ct.value_counts()

### Correlation

What might be correlated?
- postal code and popularity
- price range and popularity
- venue type and popularity
- geospatial location and popularity
...and a mix of all of the above?

Strategy:
- correlation between venue popularity vs [postal code, price range, venue type]


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
analysis_df.head(3)

In [ ]:
analysis_df.dtypes

In [ ]:
corr_data = analysis_df.copy(deep=True)
corr_data['Venue_Price_Category'] = corr_data['Venue_Price_Category'].astype('int32')

In [ ]:
corr_data[['Venue_Price_Category', 'Venue_Rank']].corr()

In [ ]:
# checking for correlation within postal code areas
corr_grouped = corr_data.groupby('Venue_Postal_Code')[['Venue_Rank', 'Venue_Price_Category']].mean().reset_index()
corr_grouped.corr()

In [ ]:
# Looking at correlation between postal code and popularity
sns.regplot(x='Venue_Price_Category', y='Venue_Rank', data=corr_grouped)
plt.ylim(0,)

Given the above, we can see there is a weak negative correlation between the ranking of venues and the mean venue price category per postal code.

### Going further

Now we have the data that we need for the analysis:
- Venue information
  - ID
  - Name
  - FourSquare Rank (popularity)
  - Price Range
  - Category
  - Location
- Location information
  - Postal Code
  - Postal Code geospatial coordinates
  - Location Name

From this information, we will investigate whether there are gaps in the market:
1. Do we have popular venues with poor rank?
2. Which venue categories do the best?
3. Are there locations that are better than others?

#### Analyzing business opportunities

Note: We might remove duplicate venues (same venue in more than one postal code) here if we want to do additional QA.

Ok, so we know have identified the most popular areas for the given business type we are looking for.

The next step is to identify the characteristics of these areas, and to find out how we can use this information to identify business opporunities.

**Model possibilities:**
- Predict the rank of a new venue based on location, price range, and venue category


**Possible business opportunities:**
1. If popular businesses have poor ratings, it would be likely that establishing a business with higher quality will attract customers.
2. If there are gaps in a given price segment, that might be popular with customes looking for a different price category.
3. If the venue category is very saturated with one category, a business within a different category might be appealing for customes looking for something different.
4. Since results are provided by FourSquare in order of popularity, we might assign a number (i.e. Rank) to each of the venues. Then we might use k-means clustering to identify the characteristics of the most popular places versus the less popular places. Features might be rank, venue category, geolocation, and possible other FourSquare (premium) features.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Using simple linear regression model
lm = LinearRegression()
X = analysis_df[['Venue_Price_Category']]
Y = analysis_df['Venue_Rank']
lm.fit(X,Y)
Yhat = lm.predict(X)
print('The intercept is', lm.intercept_)
print('The slope is', lm.coef_)

In [ ]:
# Using multiple linear regression model
#multi_lm = LinearRegression()
#Z = analysis_df[['Venue_Latitude', 'Venue_Longitude']]
#Y = analysis_df['Venue_Rank']
#multi_lm.fit(Z, Y)
#Yhat.predict()

In [ ]:
train_df = analysis_df.copy(deep=True)

In [ ]:
train_df.drop(['Postal_Code', 'Place_Name', 'State_Name', 'State_Code', 'Country_Code', 'PC_Latitude', 'PC_Longitude'], axis=1, inplace=True)
train_df.columns

In [ ]:
train_df.shape[0]

In [ ]:
len(train_df.Venue_Category.unique())

In [ ]:
train_df['Venue_Postal_Code'] = train_df['Venue_Postal_Code'].astype('category')
train_df['Venue_Category'] = train_df['Venue_Category'].astype('category')
train_df['Venue_Price_Category'] = train_df['Venue_Price_Category'].astype('category')

In [ ]:
#top = test['GarageType'].isin(test['GarageType'].value_counts().index[:5])
#test.loc[~top, 'GarageType'] = "other"
train_df['Venue_Postal_Code'].isin(train_df['Venue_Postal_Code'].value_counts().index[:5])

In [ ]:
train_df.groupby('Venue_Category')['Venue_Name'].count().sort_values(ascending=False)

In [ ]:
# Only keep postal codes with at least 10 venues
keep_pc = train_df.groupby('Venue_Postal_Code')['Venue_Name'].count().sort_values(ascending=False)>=10
keep_pc = keep_pc[keep_pc.values==True].index.values # retrieve only postal codes matching the criterion
pc_train_df = train_df.loc[train_df['Venue_Postal_Code'].isin(keep_pc)]
print('The training dataframe for postal codes have # rows:', pc_train_df.shape[0])

In [ ]:
# Only keep venue categories with at least 10 venues
keep_cat = train_df.groupby('Venue_Category')['Venue_Name'].count().sort_values(ascending=False)>=10
keep_cat = keep_cat[keep_cat.values==True].index.values # retrieve only categories matching the criterion
cat_train_df = train_df.loc[train_df['Venue_Category'].isin(keep_cat)]
print('The training dataframe for venue categories have # rows:', cat_train_df.shape[0])

In [ ]:
cat_train_df.head(5)

In [ ]:
train_df.dtypes

### Machine Learning: Applying label encoding to price categories to train a model

In [ ]:
y_train = cat_train_df['Venue_Rank']
y_train

In [ ]:
X_train = cat_train_df['Venue_Price_Category']

In [ ]:
X_train.dtypes == 'category'

In [ ]:
object_cols = X_train.name

### Clustering venues

Let's do some testing on venue clustering. First one-hot encoding them, and then see what results we get.

**First: Cluster venues according to the mean rank in per price range in each postal code**

In [ ]:
# The dataset we're working with
#train_df = analysis_df.copy(deep=True)
#train_df = train_df[['']]
#train_df.drop(['Postal_Code', 'Place_Name', 'State_Name', 'State_Code', 'Country_Code', 'PC_Latitude', 'PC_Longitude'], axis=1, inplace=True)
#train_df.head(3)
analysis_df.head(3)

In [ ]:
analysis_df['Venue_Postal_Code'] = analysis_df['Venue_Postal_Code'].astype('category')
analysis_df['Venue_Category'] = analysis_df['Venue_Category'].astype('category')
analysis_df['Venue_Price_Category'] = analysis_df['Venue_Price_Category'].astype('category')

In [ ]:
analysis_df[['Venue_Postal_Code', 'Venue_Category']].head(3)

In [ ]:
# Calculating the average rank per postal code and price category
price_rating = analysis_df.groupby(['Venue_Postal_Code', 'Venue_Price_Category'])['Venue_Rank'].mean().reset_index()
price_rating

In [ ]:
def cluster_postal(postal_code, price_category, venue_rank):
  """Function to cluster a given postal code area"""

In [ ]:
# Ranking venues with 1-indexation for later on
analysis_df['Venue_Rank'] = list(range(1,analysis_df.shape[0]+1))

In [ ]:
analysis_df.head(3)

In [ ]:
analysis_df.isna

In [ ]:
# Simple one-hot encoding: Venues per price category
price_cat_onehot = pd.get_dummies(analysis_df['Venue_Price_Category'], prefix='', prefix_sep='')
price_cat_onehot.head(3)

In [ ]:
analysis_df.loc[(analysis_df['Venue_Postal_Code']=='89014') & (analysis_df['Venue_Price_Category']==4)]

In [ ]:
price_cat_onehot = pd.get_dummies(price_rating['Venue_Price_Category'], prefix="", prefix_sep="")
price_cat_onehot = pd.concat([analysis_df[['Venue_Postal_Code', 'Venue_Rank']], price_cat_onehot], axis=1)
price_cat_onehot.head(5)

In [ ]:
price_cat_onehot.isna

In [ ]:
price_cat_onehot[['1', '2', '3', '4']].astype('int32')

In [ ]:
price_cat_onehot.dtypes

In [ ]:
price_cat_onehot.groupby(['Venue_Postal_Code']).count()

In [ ]:
# grouping the postal codes accoring to mean number of venue ratings
price_cat_grouped = price_cat_onehot.groupby('Venue_Postal_Code').count().reset_index()
price_cat_grouped

In [ ]:
# Grouping the neighborhoods by mean number of the various venue categories
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

# Showing the mean occurrence of venue categories per neighborhood
toronto_grouped

### Model evaluation and refinement

In [ ]:
from sklearn.model_selection import train_test_split

x_data = cat_train_df['Venue_Price_Category']
y_data = cat_train_df['Venue_Rank']

x_train, x_test, y_train, y_test = train_test_split(x_data,
                                                    y_data,
                                                    test_size = 0.3, #30% of dataset
                                                    random_state = 0) # random seed for sampling

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import linear_model

lr = linear_model.LinearRegression()

# initialize a linear regression model, using x_data and y_data, with 3 partitions
scores = cross_val_score(lr, x_data, y_data, cv=3)
np.mean(scores)